In [ ]:
# Import required libraries and dependencies
import pandas as pd
import hvplot.pandas
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Load the data into a Pandas DataFrame
df_market_data = pd.read_csv(
    "Resources/crypto_market_data.csv",
    index_col="coin_id")

# Display sample data
df_market_data.head(10)

In [ ]:
# Generate summary statistics
df_market_data.describe()

In [ ]:
# Plot your data to see what's in your DataFrame
df_market_data.hvplot.line(
    width=800,
    height=400,
    rot=90
)

---

### Prepare the Data

In [ ]:
# Use the `StandardScaler()` module from scikit-learn to normalize the data from the CSV file
scaler = StandardScaler()

# create columns and index list for scaled transformation df
columns = list(df_market_data.columns)
index = list(df_market_data.index.values)
index_name = df_market_data.index.name

# scale and verify data
scaled_values =scaler.fit_transform(df_market_data)
scaled_values[:5]
columns
index[:5]
index_name

In [ ]:
# Create a DataFrame with the scaled data
# Copy the crypto names from the original data
# Set the coinid column as index
# Display sample data

# convert scaled data into a new dataframe
market_scaled_df = pd.DataFrame(
    scaled_values,
    columns = columns,
    index = index
)
market_scaled_df.index.name = index_name
market_scaled_df.head()

---

### Find the Best Value for k Using the Original Data.

In [ ]:
# Create an empty list to store the inertia values
# Create a list with the number of k-values from 1 to 11
# Create a a list to store inertia values and the values of k
inertia = []
k = list(range(1, 11))

In [ ]:
# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
# 2. Fit the model to the data using `df_market_data_scaled`
# 3. Append the model.inertia_ to the inertia list
for i in k:
    k_model = KMeans(n_clusters=i)
    k_model.fit(market_scaled_df)
    inertia.append(k_model.inertia_)

In [ ]:
# Create a dictionary with the data to plot the Elbow curve
elbow_data = {"k": k, "inertia": inertia}

# Create a DataFrame with the data to plot the Elbow curve
df_elbow = pd.DataFrame(elbow_data)

df_elbow

In [ ]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.
ms_elbow_plot = df_elbow.hvplot.line(
    x="k", 
    y="inertia", 
    title="Elbow Curve", 
    xticks=k
)
ms_elbow_plot

#### Answer the following question: 

**Question:** What is the best value for `k`?

**Answer:** 4 appears to be the best value. After 4 the rate of inertia decrease slows considerably.

---

### Cluster Cryptocurrencies with K-means Using the Original Data

In [ ]:
# Initialize the K-Means model using the best value for k
model = KMeans(n_clusters=4)

In [ ]:
# Fit the K-Means model using the scaled data
model.fit(market_scaled_df)

In [ ]:
# Predict the clusters to group the cryptocurrencies using the scaled data
k_4 = model.predict(market_scaled_df)

# Print the resulting array of cluster values.
k_4

In [ ]:
# Create a copy of the DataFrame
market_scaled_df_predict = market_scaled_df.copy()

In [ ]:
# Add a new column to the DataFrame with the predicted clusters
market_scaled_df_predict['segment'] = k_4

# Display sample data
market_scaled_df_predict.head()

In [ ]:
# Create a scatter plot using hvPlot by setting 
# `x="price_change_percentage_24h"` and `y="price_change_percentage_7d"`. 
# Color the graph points with the labels found using K-Means and 
# add the crypto name in the `hover_cols` parameter to identify 
# the cryptocurrency represented by each data point.

ms_predict_plot = market_scaled_df_predict.hvplot.scatter(
    x="price_change_percentage_24h",
    y="price_change_percentage_7d",
    xlabel = 'PC1',
    ylabel = 'PC2',
    by="segment",
    hover_cols = 'coin_id'
)
ms_predict_plot

---

### Optimize Clusters with Principal Component Analysis.

In [ ]:
# Create a PCA model instance and set `n_components=3`.
pca = PCA(n_components=3)

In [ ]:
# Use the PCA model with `fit_transform` to reduce to 
# three principal components.
market_scaled_pca_df = market_scaled_df.copy()
market_pca = pca.fit_transform(market_scaled_pca_df)

market_pca_df = pd.DataFrame(
    market_pca,
    columns=["PCA1", "PCA2", "PCA3"]
)

# View the first five rows of the DataFrame. 
market_pca_df.head()

In [ ]:
# Retrieve the explained variance to determine how much information 
# can be attributed to each principal component.
exp_var = pca.explained_variance_ratio_
display(exp_var)
total_exp_var = sum(exp_var)
display(total_exp_var)

#### Answer the following question: 

**Question:** What is the total explained variance of the three principal components?

**Answer:** Total explained variance is 0.895

In [ ]:
# Create a new DataFrame with the PCA data
# Creating a DataFrame with the PCA data
# Copy the crypto names from the original data

# DF already created in above cell, just adding index and index_name
market_pca_df.index = index
market_pca_df.index.name = index_name

# Display sample data
market_pca_df.head()

---

### Find the Best Value for k Using the PCA Data

In [ ]:
# Create a list with the number of k-values from 1 to 11

k = list(range(1, 11))

In [ ]:
# Create an empty list to store the inertia values
inertia = []

# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
# 2. Fit the model to the data using `df_market_data_pca`
# 3. Append the model.inertia_ to the inertia list
for i in k:
    k_model_pca = KMeans(n_clusters=i)
    k_model_pca.fit(market_pca_df)
    inertia.append(k_model_pca.inertia_)

In [ ]:
# Create a dictionary with the data to plot the Elbow curve
elbow_data_pca = {"k": k, "inertia": inertia}

# Create a DataFrame with the data to plot the Elbow curve
df_elbow_pca = pd.DataFrame(elbow_data_pca)
df_elbow_pca

In [ ]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.
pca_elbow_plot = df_elbow_pca.hvplot.line(
    x="k", 
    y="inertia", 
    title="Elbow Curve", 
    xticks=k
)
pca_elbow_plot

#### Answer the following questions: 

* **Question:** What is the best value for `k` when using the PCA data?

  * **Answer:** 4 appears to be the best value. After 4 the rate of inertia decrease slows considerably.


* **Question:** Does it differ from the best k value found using the original data?

  * **Answer:** It does not.

### Cluster Cryptocurrencies with K-means Using the PCA Data

In [ ]:
# Initialize the K-Means model using the best value for k
# Initialize the K-Means model using the best value for k
model_pca = KMeans(n_clusters=4)

In [ ]:
# Fit the K-Means model using the PCA data
# Fit the K-Means model using the scaled data
model_pca.fit(market_pca_df)

In [ ]:
# Predict the clusters to group the cryptocurrencies using the scaled data
k_4_pca = model_pca.predict(market_pca_df)

# changing default segment ids for better comparison with composite plots
k_4_pca = [x + 4 for x in k_4_pca]

# Print the resulting array of cluster values.
k_4_pca[:6]

In [ ]:
# Create a copy of the DataFrame with the PCA data
market_pca_df_predict = market_pca_df.copy()

# Add a new column to the DataFrame with the predicted clusters
market_pca_df_predict['segment'] = k_4_pca

# Display sample data
market_pca_df_predict.head()

In [ ]:
# Create a scatter plot using hvPlot by setting 
# `x="PCA1"` and `y="PCA2"`. 
# Color the graph points with the labels found using K-Means and 
# add the crypto name in the `hover_cols` parameter to identify 
# the cryptocurrency represented by each data point.
pca_predict_plot = market_pca_df_predict.hvplot.scatter(
    x="PCA1",
    y="PCA2",
    xlabel = 'PC1',
    ylabel = 'PC2',
    by="segment",
    hover_cols = 'coin_id'
)
pca_predict_plot

### Visualize and Compare the Results

In this section, you will visually analyze the cluster analysis results by contrasting the outcome with and without using the optimization techniques.

In [ ]:
# Composite plot to contrast the Elbow curves

# homework instructions say to use(+) but (*) is needed to create a composite plot
# (+) creates sid-by-side plots

ms_elbow_plot * pca_elbow_plot

In [ ]:
# Composite plot to contrast the clusters
# display(ms_predict_plot)
# display(pca_predict_plot)
pca_predict_plot * ms_predict_plot

#### Answer the following question: 

  * **Question:** After visually analyzing the cluster analysis results, what is the impact of using fewer features to cluster the data using K-Means?

  * **Answer:** Using fewer features had little overall effect on clustering (segments). Clusters were divided very similarly between full-data and PCA plots both in size and membership.